# 2.6: 约化密度矩阵（Reduced Density Matrix）

量子态的"局域"性质：

例如，对于4比特量子态，其编号为0的量子位关于单体算符 $\hat{\sigma}^{(\alpha)}$ 的量子期望

![reduced_density_matrix_equation](./images/reduced_density_matrix_equation.png)

$\langle\psi|\hat{\sigma}^{(\alpha)}|\psi\rangle = \sum_{s_0s_1s_2s_3}\sum_s \sigma_{s_0s}^{(\alpha)}\psi_{s_0s_1s_2s_3}^*\psi_{ss_1s_2s_3} = \sum_{ss_0} \sigma_{s_0s}^{(\alpha)}\rho_{ss_0}$

> 先对 $s_1s_2s_3$ 求和，可以得到 $\rho_{ss_0}$

通过整理上式易得关于编号为0量子位的约化密度算符（的系数矩阵）满足

$\rho^{[0]} = \psi_{[0]}\psi_{[0]}^{\dagger}$

于是有：$\langle\psi|\hat{\sigma}^{(\alpha)}|\psi\rangle = \text{Tr}(\hat{\rho}^{[0]}\hat{\sigma}^{(\alpha)})$

> 推导:
> 
> $\sum_{ss_0} \sigma_{s_0s}^{(\alpha)}\rho_{ss_0} = \sum_{s_0} \sum_{s} \sigma_{s_0s}^{(\alpha)}\rho_{ss_0}$
> 
> 1. 对 $s$ 求和即为向量乘法，选的是 $\sigma$ 的第 $s_0$ 行与 $\rho$ 的第 $s_0$ 列的内积，得到 $(s_0, s_0)$ 位置的元素
> 2. 加总起来，就是迹


约化密度算符的一般形式（对于第 i, j, k ... 个量子位的约化密度算符）：

算符：$\hat{\rho}^{[ijk...]} = \text{Tr}_{/ijk...}(|\psi\rangle\langle\psi|)$

> 除开 i, j, k ... 之外，求迹

系数张量：$\rho^{[ijk...]} = \psi_{[ijk...]}\psi_{[ijk...]}^{\dagger}$

> 把 i, j, k ... 指标作为左指标，其余作为右指标，展开成矩阵

算符的期望值等于相应约化密度算符与其乘积后的迹。

约化密度算符的概率满足：

$p_n = \langle \phi^{(n)}|\hat{\rho}^{[ijk...]}|\phi^{(n)}\rangle = \text{Tr}(\hat{\rho}^{[ijk...]}|\phi^{(n)}\rangle\langle \phi^{(n)}|) = \langle\hat{P}\rangle$

> 这里 n 代表这 i,j,k ... 子体系里面的第 n 个状态/基底

> TODO: 第二个等号为什么成立？

为投影算符$\hat{P} = |\phi^{(n)}\rangle\langle \phi^{(n)}|$的量子期望

> 例：4比特量子态$|\psi\rangle$并在编号为1与2的量子位上采样，获得$|00\rangle$的概率为
> 
> $p_{00} = \langle 00|\hat{\rho}^{[12]}|00\rangle$

约化密度算符的迹归一化对应于概率归一化

$\sum_n p_n = \sum_n\langle\phi^{(n)}|\hat{\rho}^{[ijk...]}|\phi^{(n)}\rangle =\sum_n\text{Tr}(\hat{\rho}^{[ijk...]}|\phi^{(n)}\rangle\langle \phi^{(n)}|)= \text{Tr}(\hat{\rho}^{[ijk...]}) = 1$

> 根据热态的定义，约化密度算符也是一个热态

纯态的任意约化密度有：$\text{Tr}(\hat{\rho}^{[ijk...]}) = \langle\psi|\psi\rangle = 1$

## Code

In [1]:
# |default_exp quantum_state.functional
# |export
from typing import List
import math
import torch
from tensor_network.utils import check_quantum_gate

In [2]:
# |export
def calc_reduced_density_matrix(state: torch.Tensor, qubit_idx: int | List[int]):
    assert isinstance(qubit_idx, (int, list)), "qubit_idx must be an integer or a list of integers"
    if isinstance(qubit_idx, int):
        qubit_idx = [qubit_idx]

    num_qubits = state.ndim
    for qi in qubit_idx:
        assert 0 <= qi < num_qubits, "qubit_idx must be in [0, num_qubits - 1]"

    qubit_indices = list(range(num_qubits))
    dims_to_reduce = [i for i in qubit_indices if i not in qubit_idx]
    dims_to_keep = qubit_idx
    state = state.permute(*(dims_to_keep + dims_to_reduce))  # (*dims_to_keep, *dims_to_reduce)
    shape_to_keep = math.prod([state.shape[i] for i in dims_to_keep])
    shape_to_reduce = math.prod([state.shape[i] for i in dims_to_reduce])
    state = state.reshape(shape_to_keep, shape_to_reduce)
    reduced_density_matrix = state @ state.conj().T
    return reduced_density_matrix


def calc_observation(
    state: torch.Tensor, operator: torch.Tensor, qubit_idx: int | List[int], _fast_mode: bool = True
):
    if isinstance(qubit_idx, int):
        length = 1
    else:
        length = len(qubit_idx)
    reduced_density_matrix = calc_reduced_density_matrix(state, qubit_idx)
    num_qubits_operator = check_quantum_gate(operator)
    assert num_qubits_operator == length, (
        "The number of qubits of the operator does not match the number of qubits of the state"
    )
    operator_mat = operator.reshape(2**num_qubits_operator, 2**num_qubits_operator)
    if _fast_mode:
        return (reduced_density_matrix * operator_mat.T).sum()
    else:
        return torch.trace(reduced_density_matrix @ operator_mat)

### Test

In [3]:
from tensor_network import setup_ref_code_import
from Library.QuantumState import TensorPureState
from itertools import combinations

From setup_ref_code_import:
  Added reference_code_path='/Users/zhiqiu/offline_code/personal/tensor_network/reference_code' to sys.path.
  You can import the reference code now.


In [4]:
num_qubits = 5

state = TensorPureState(nq=num_qubits, dtype=torch.complex128)
tensor = state.tensor

reduced_density_matrix_int = state.reduced_density_matrix(0)
reduced_density_matrix_list = state.reduced_density_matrix([0])

qubit_indices = list(range(num_qubits))

for length in range(1, num_qubits):
    for comb in combinations(qubit_indices, length):
        comb = list(comb)
        reduced_density_matrix = calc_reduced_density_matrix(tensor, comb)
        reduced_density_matrix_ref = state.reduced_density_matrix(comb)
        assert torch.allclose(reduced_density_matrix, reduced_density_matrix_ref), (
            f"{(reduced_density_matrix - reduced_density_matrix_ref).norm()}"
        )

In [5]:
num_qubits = 5

state = TensorPureState(nq=num_qubits, dtype=torch.complex128)
tensor = state.tensor

qubit_indices = list(range(num_qubits))

for length in range(1, num_qubits):
    for comb in combinations(qubit_indices, length):
        comb = list(comb)
        operator = torch.randn(*([2] * (length * 2)), dtype=torch.complex128)
        observation = calc_observation(tensor, operator, comb)
        observation_ref = state.observation(operator, comb)
        assert torch.allclose(observation, observation_ref), (
            f"{(observation - observation_ref).norm()}"
        )

# 2.6: 量子纠缠

### 纠缠谱

纠缠谱计算的是二分的子体系相互纠缠的程度

纠缠谱计算方法：
1. 矩阵化：把系数张量矩阵化，一个子体系对应的 index 变成左指标，另一个子体系对应的 index 变成右指标
2. 奇异值分解：获得这个矩阵的奇异值谱

> 另一种计算方法：奇异值谱的平方由对应约化密度算符系数矩阵的本征谱给出

纠缠熵计算方法：
1. 计算纠缠谱
2. 以纠缠谱的平方作为概率分布
3. 计算熵

TODO: 看书 2.6 节
